# Passing Static Parameters

Aim: pass parameters to the cycle components, when they are needed.


In [1]:
import numpy as np
import pandas as pd
from autora.experimentalist.pipeline import make_pipeline
from autora.variable import VariableCollection, Variable
from sklearn.linear_model import LinearRegression

from autora.workflow import Cycle

Here we have an experimentalist which takes a parameter:

In [2]:
uniform_random_rng = np.random.default_rng(180)
def uniform_random_sampler(n):
    new_conditions = uniform_random_rng.uniform(low=0, high=11, size=n)
    conditions_df = pd.DataFrame({"x": new_conditions})
    return conditions_df
example_experimentalist_with_parameters = make_pipeline([uniform_random_sampler])

In [3]:
example_experimentalist_with_parameters(**{"uniform_random_sampler": {"n": 1}})

,x
0,6.33662


In [4]:
def ground_truth(x):
    return x + 1
metadata = VariableCollection(
   independent_variables=[Variable(name="x", allowed_values=range(11))],
   dependent_variables=[Variable(name="y", value_range=(-20, 20))],
   )
example_experimentalist = make_pipeline([metadata.independent_variables[0].allowed_values])

def get_example_synthetic_experiment_runner():
    rng = np.random.default_rng(seed=180)
    def runner(x):
        return ground_truth(x) + rng.normal(0, 0.1, x.shape)
    def as_dataframe(conditions_df: pd.DataFrame):
        observations_df = conditions_df.copy()
        observations_df["y"] = runner(conditions_df["x"])
        return observations_df
    return as_dataframe

example_synthetic_experiment_runner = get_example_synthetic_experiment_runner()
example_theorist = LinearRegression()

The cycle can handle that using the `params` keyword:

In [5]:
cycle_with_parameters = Cycle(
    metadata=metadata,
    theorist=example_theorist,
    experimentalist=example_experimentalist_with_parameters,
    experiment_runner=example_synthetic_experiment_runner,
    params={"experimentalist": {"uniform_random_sampler": {"n": 7}}}
)
cycle_with_parameters.run()
cycle_with_parameters.data.conditions[-1]

,x
0,7.349166
1,6.085965
2,2.285666
3,1.955397
4,5.800231
5,3.270079
6,10.583823


For the next cycle, if we wish, we can change the parameter value:

In [6]:
cycle_with_parameters.params["experimentalist"]["uniform_random_sampler"]["n"] = 2
cycle_with_parameters.run()
cycle_with_parameters.data.conditions[-1]


,x
0,9.456660
1,2.217216
